In [2]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py - Exponential Smoothing Solution

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Read data from the provided URLs
TRAIN_URL = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv"
TEST_URL = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_test.csv"

# Load and preprocess the training data
train_df = pd.read_csv(TRAIN_URL, parse_dates=["Timestamp"])
train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")

# Process the target variable - handle any data issues
y_train = pd.to_numeric(train_df["trips"], errors="coerce")
# Fill any remaining missing values using time-based interpolation
y_train = y_train.interpolate(method="time", limit_direction="both")

# Create the Exponential Smoothing model (unfitted)
# Using additive seasonality with 24-hour period
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated"
)

# Fit the model to the training data
modelFit = model.fit(optimized=True, use_brute=True)

# Load the test data
test_df = pd.read_csv(TEST_URL, parse_dates=["Timestamp"])

# Generate forecasts for the test period (744 hours for January 2019)
h = len(test_df)
forecast = modelFit.forecast(steps=h)

# Ensure predictions are in the correct format for the autograder
# Convert to numpy array, ensure it's 1D, and ensure no negative values
pred = np.asarray(forecast, dtype=float).ravel()
pred = np.maximum(pred, 0)  # Ensure no negative values

# For debugging: check the output format
print(f"Prediction length: {len(pred)}")
print(f"Prediction type: {type(pred)}")
print(f"First few values: {pred[:5]}")

/tmp/ipython-input-2988366806.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")


Prediction length: 744
Prediction type: <class 'numpy.ndarray'>
First few values: [5606.1097574  3577.85981238 2565.74546374 2151.00488427 2202.56732959]
